# Run all 

In [ ]:
import sys, os
import pandas as pd
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), ".."))) # lägg till training i path
from src.data_loader import load_local_data, unload_local_data
from src.scrapers.abstract_scraper import AbstractScraper
from src.scrapers.afry_scraper import AfryScraper
from src.scrapers.aliant_scraper import AliantScraper
from src.scrapers.asociety_scraper import ASocietyScraper
from src.scrapers.cinode_scraper import CinodeScraper
from src.scrapers.combitech_scraper import CombitechScraper
from src.scrapers.emagine_scraper import EmagineScraper
from src.scrapers.ework_scraper import EworkScraper
from src.scrapers.itcnetwork_scraper import ITCNetworkScraper
from src.scrapers.nikita_scraper import NikitaScraper
from src.scrapers.profinder_scraper import ProfinderScraper
from src.scrapers.regent_scraper import RegentScraper
from src.scrapers.senterprise_scraper import SenterpriseScraper
from src.scrapers.sigma_scraper import SigmaScraper
from src.scrapers.tingent_scraper import TingentScraper
from src.scrapers.upgraded_scraper import UpgradedScraper


scrapers = [AfryScraper(), AliantScraper(), ASocietyScraper(), CinodeScraper(), CombitechScraper(), EmagineScraper(), EworkScraper(), ITCNetworkScraper(), NikitaScraper(), ProfinderScraper(), RegentScraper(), SenterpriseScraper(), SigmaScraper(), TingentScraper(), UpgradedScraper()]
#scrapers = [CinodeScraper()]
#scrapers = []

nr_ads_pre = len(load_local_data())

for s in scrapers:
    response = s.request_status()
    scraped_payload_dict = s.scrape_jobs_payloads_dict(response)

    old_payload_data = load_local_data(file_name='jobs_payloads.csv')
    old_payload_dict = dict(zip(old_payload_data['id'], old_payload_data['raw_payload']))
    old_bronze_data = load_local_data()
    
    new_payload_dict = s.return_new_payloads(new_dict=scraped_payload_dict, old_dict=old_payload_dict)
    new_bronze_data = s.parse_bronze_data(new_payload_dict)

    updated_payload_dict = s.concat_dicts(new_payload_dict, old_payload_dict)
    updated_payload_data = pd.DataFrame(list(updated_payload_dict.items()), columns=['id','raw_payload'])
    updated_bronze_data = s.concat_new_rows(new_bronze_data, old_bronze_data)

    #unload_local_data(updated_payload_data, file_name='jobs_payloads.csv')
    #unload_local_data(updated_bronze_data)

nr_ads_post = len(pd.read_csv('../data/bronze/jobs.csv'))
print('Total number of new added jobs:', nr_ads_post-nr_ads_pre)
new_bronze_data


Afry > Response: 200
Afry > Nmr of scraped adds: 74
Afry > Parsing bronze data: 8
Aliant > Response: 200
Aliant > Nmr of scraped adds: 9
Aliant > Parsing bronze data: 0
A Society > Response: 200
A Society > Nmr of scraped adds: 117
A Society > Parsing bronze data: 11
Cinode > Response: 200
Cinode > Nmr of scraped adds: 20
Cinode > Parsing bronze data: 20
Combitech > Response: 200
Combitech > Nmr of scraped adds: 35
Combitech > Parsing bronze data: 9
Emagine > Response: 200
Emagine > Nmr of scraped adds: 49
Emagine > Parsing bronze data: 4
Ework > Response: 200
Ework > Nmr of scraped adds: 97
Ework > Parsing bronze data: 33
ITC Network > Response: 200
ITC Network > Nmr of scraped adds: 42
ITC Network > Parsing bronze data: 42
Nikita > Response: 200
Nikita > Nmr of scraped adds: 20
Nikita > Parsing bronze data: 1
Profinder > Response: 200
Profinder > Nmr of scraped adds: 24
Profinder > Parsing bronze data: 24
Regent > Response: 200
Regent > Nmr of scraped adds: 34
Regent > Parsing bronze

,id,site,site_id,job_title,area,due_date,work_location,work_type,link,ingestion_ts,is_new
0,Upgraded-https://upgraded.se/konsultuppdrag/12...,Upgraded,https://upgraded.se/konsultuppdrag/129483-seni...,Senior .Net – Papyrus,.NET-utvecklare,2025-09-24,Stockholm,,https://upgraded.se/konsultuppdrag/129483-seni...,2025-09-23 17:44:10,True
1,Upgraded-https://upgraded.se/konsultuppdrag/12...,Upgraded,https://upgraded.se/konsultuppdrag/129473-main...,Mainframeutvecklare – Försäkring/Liv & Pension,Mainframeutvecklare,2025-10-14,Stockholm,"Distansarbete: Hybrid, 3 dagar/vecka onsite",https://upgraded.se/konsultuppdrag/129473-main...,2025-09-23 17:44:10,True
2,Upgraded-https://upgraded.se/konsultuppdrag/12...,Upgraded,https://upgraded.se/konsultuppdrag/129469-fron...,Frontendutvecklare med UX/UI-kompetens,Frontendutvecklare,2025-09-25,"Lund, SE",,https://upgraded.se/konsultuppdrag/129469-fron...,2025-09-23 17:44:10,True
3,Upgraded-https://upgraded.se/konsultuppdrag/12...,Upgraded,https://upgraded.se/konsultuppdrag/129461-seni...,Senior Network / Check Point Specialist,Network Specialist,2025-11-28,Stockholm,,https://upgraded.se/konsultuppdrag/129461-seni...,2025-09-23 17:44:10,True
4,Upgraded-https://upgraded.se/konsultuppdrag/12...,Upgraded,https://upgraded.se/konsultuppdrag/129455-verk...,Verksamhetsutvecklare Hemtjänst – Ersättningar...,Verksamhetsutvecklare,2025-09-29,"Solna, SE","Distansarbete: Hybridarbete enl ök, normal arb...",https://upgraded.se/konsultuppdrag/129455-verk...,2025-09-23 17:44:10,True
5,Upgraded-https://upgraded.se/konsultuppdrag/12...,Upgraded,https://upgraded.se/konsultuppdrag/129421-serv...,ServiceNow-Utvecklare,Utvecklare,2025-09-29,Stockholm,Distansarbete: Hybrid: 2-3 dagar onsite,https://upgraded.se/konsultuppdrag/129421-serv...,2025-09-23 17:44:10,True
6,Upgraded-https://upgraded.se/konsultuppdrag/12...,Upgraded,https://upgraded.se/konsultuppdrag/129414-test...,Testledare,Testledare,2025-09-29,Karlstad,,https://upgraded.se/konsultuppdrag/129414-test...,2025-09-23 17:44:10,True
7,Upgraded-https://upgraded.se/konsultuppdrag/12...,Upgraded,https://upgraded.se/konsultuppdrag/129384-ekonom/,Ekonom,Ekonom,2025-09-29,"Stockholm, SE",Distansarbete: Hybrid,https://upgraded.se/konsultuppdrag/129384-ekonom/,2025-09-23 17:44:10,True
8,Upgraded-https://upgraded.se/konsultuppdrag/12...,Upgraded,https://upgraded.se/konsultuppdrag/129383-pene...,Penetrationstestare 2025,Säkerhetsspecialist,2025-10-10,"Haninge/Distans, SE",Distansarbete: Uppstartsmöte på plats i övrigt...,https://upgraded.se/konsultuppdrag/129383-pene...,2025-09-23 17:44:10,True
9,Upgraded-https://upgraded.se/konsultuppdrag/12...,Upgraded,https://upgraded.se/konsultuppdrag/129379-tekn...,Teknisk Projektledare – inom infrastruktur,Teknisk projektledare,2025-10-02,Sundsvall/Stockholm,Distansarbete: Hybrid,https://upgraded.se/konsultuppdrag/129379-tekn...,2025-09-23 17:44:10,True


In [28]:
def write_html(html,name='one_job'):
    with open(f'{name}.html', "w", encoding="utf-8") as f:
        f.write(str(html))



In [77]:
from playwright.sync_api import sync_playwright
from bs4 import BeautifulSoup

import re
import requests
from bs4 import BeautifulSoup
import re
import requests


class ProfinderScraper:

    def clean_title(self, title: str):
        """Remove 'ID:xxx' from title."""
        return re.sub(r"\s*ID:\d+", "", title).strip()

    def extract_id(self, title: str):
        """Extract job ID if present."""
        match = re.search(r"ID:(\d+)", title)
        return match.group(1) if match else None

    def extract_location(self, title: str):
        """Try to extract location if mentioned with 'i <Ort>'."""
        match = re.search(r" i ([A-ZÅÄÖa-zåäö\s-]+)", title)
        return match.group(1).strip() if match else None


    def scrape_all_jobs(self, max_pages: int = 5):
        all_jobs = []
        for page in range(1, max_pages + 1):
            data = self.get_page(page)
            posts = data.get("postFeedPage", {}).get("posts", {}).get("posts", [])
            if not posts:
                break
            for post in posts:
                title = post.get("title", "")
                job = {
                    "id": self.extract_id(title),
                    "title": self.clean_title(title),
                    "location": self.extract_location(title),
                    "excerpt": post.get("excerpt"),
                    "url": post.get("link"),
                    "published": post.get("firstPublishedDate"),
                }
                all_jobs.append(job)
            print(f"{self.__class__.site} > Scraped {len(posts)} jobs from page {page}")
        return all_jobs





## Load bronze table 

In [17]:
import pandas as pd
from src.data_loader import load_local_data, unload_local_data
bronze_data = load_local_data()
df = bronze_data.copy()
print(len(df['work_location'].unique()))
df['work_location'].unique()


123


array(['Göteborg', 'Jönköping', 'Kalmar län', 'Södermanlands län',
       'Stockholms län', 'Skåne län', 'Västra Götalands län',
       'Västernorrlands län Västerbottens län',
       'Västerbottens län Västernorrlands län', 'Östergötlands län',
       'Blekinge län', 'Västernorrlands län', nan, 'Västmanlands län',
       'Västmanlands län Stockholms län Uppsala län',
       'Dalarnas län Västra Götalands län Västmanlands län',
       'Norrbottens län', 'Örebro län', 'Stockholms län Kalmar län',
       'Stockholms län Uppsala län', 'Kalmar län Stockholms län',
       'Stockholms län Norrbottens län Västernorrlands län Västra Götalands län',
       'Västmanlands län Västernorrlands län Stockholms län',
       'Östergötlands län Gävleborgs län Dalarnas län',
       'Stockholms län Södermanlands län', 'Stockholm', 'Malmö/Lund',
       'Södertälje', 'Linköping', 'Boden', 'Karlskoga', 'Karlskrona',
       'Växjö', 'Västerås', 'Ludvika', 'Norrköping', 'Finspång',
       'Uppsala', 'Örebro', 

In [53]:
%pip install unidecode
%pip install rapidfuzz
%pip install geonamescache


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 51.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:

from src.transformers import clean_work_location_column

df = bronze_data.copy()
df['work_location'] = clean_work_location_column(df['work_location'])

print(len(df['work_location'].unique()))
for city in list(sorted(df['work_location'].unique())): print(city)

96

Arboga
Arboga...
Blekinge län
Boden
Borlänge
Borås
Dalarnas län
Dalarnas län...
Enköping, Stockholm
Eskilstuna
Eskilstuna, Västerås
Falkenberg
Finspång
Frösön
Gävle
Gävle, Skutskär
Gävle...
Götalands län, Västra frölunda
Götalands län...
Göteborg
Göteborg, Stockholm
Göteborg...
Hallstahammar
Halmstad
Haninge
Helsingborg
Helsinki
Helsinki, Stockholm
Huskvarna
Hässleholm
Jonsered
Järfälla
Jönköping
Kalmar
Kalmar, Stockholm
Karlskoga
Karlskrona
Karlskrona, Växjö
Karlstad, Stockholm
Katowice, Stockholm
Kista
Kungsbacka
Köpenhamn
Köping
Landskrona
Lidingö
Linköping
Linköping, Norrköping
Ludvika
Luleå
Lund
Lund, Malmö
Malmö
Malmö, Växjö
Mölndal
Nacka
Nearshore
Norrbottens län
Norrköping
Norrköping, Stockholm
Remote
Remote, Stockholm
Remote, Södertälje
Sjöbo
Skåne
Skövde, Växjö
Sollentuna
Solna
Stockholm
Stockholm, Södermanlands län
Stockholm, Uppsala
Stockholm, Visby
Stockholm, Örebro
Stockholm...
Sundbyberg
Sundsvall
Söderhamn
Södermanlands län
Södertälje
Trollhättan
Tumba
Täby
Umeå
Upp